# Necessary imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import warnings
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import plotly.express as px
import folium

warnings.filterwarnings("ignore")

# Importing data and adding features

In [2]:
file_path = 'ChallengeXHEC23022024_v1.xlsx'
excel_data = pd.read_excel(file_path, sheet_name=None)
for sheet_name, df in excel_data.items():
    
    custom_name = f'{sheet_name}_df'
    
    globals()[custom_name] = df

JAN24_df = JAN24_df[~JAN24_df['Prestation'].isin(['ADMINISTRATION', 'FORMATION', 'COORDINATION', 
                                                  'HOMMES TOUTES MAINS', 'VISITE MEDICALE'])]

### Day type feature: week-day or week-end

In [3]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

def get_day_type(date):
    if date.weekday() < 5:  # Monday to Friday
        return 'Week-day'
    else:  # Saturday or Sunday
        return 'Week-end'

JAN24_df['Day type'] = JAN24_df['Date'].apply(get_day_type)

### Day time feature : morning, afternoon, evening

In [4]:
JAN24_df['Heure de début'] = pd.to_datetime(JAN24_df['Heure de début'], format='%H:%M:%S')

def categorize_time(hour):
    if hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    else:
        return "Evening"

JAN24_df['Hour'] = JAN24_df['Heure de début'].dt.hour
JAN24_df['Day time'] = JAN24_df['Hour'].apply(categorize_time)

JAN24_df.drop(columns=['Hour'], inplace=True)


### Duration feature in minutes

In [5]:
JAN24_df['Heure de début'] = pd.to_datetime(JAN24_df['Heure de début'], format='%H:%M:%S')
JAN24_df['Heure de fin'] = pd.to_datetime(JAN24_df['Heure de fin'], format='%H:%M:%S')

JAN24_df['Duration'] = (JAN24_df['Heure de fin'] - JAN24_df['Heure de début']).dt.total_seconds() / 60

# First take on constraints

### Beginning and end hours distribution

In [6]:
def time_to_hours(t):
    return t.hour

JAN24_df['Heure de fin'] = JAN24_df['Heure de fin'].apply(time_to_hours)
JAN24_df['Heure de début'] = JAN24_df['Heure de début'].apply(time_to_hours)

prestations = JAN24_df['Prestation'].unique()

for prest in prestations:
    prest_df = JAN24_df[JAN24_df['Prestation'] == prest]
    
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f'Heure de début',
                                                       f'Heure de fin'))

    
    fig.add_trace(go.Histogram(x=prest_df['Heure de début'], marker_color='orchid', opacity=0.7, showlegend=False), row=1, col=1)

    
    fig.add_trace(go.Histogram(x=prest_df['Heure de fin'], marker_color='mediumorchid', opacity=0.7, showlegend=False), row=1, col=2)

    
    fig.update_layout(title=f'Distribution of Heure de début and Heure de fin for Prestation {prest}',
                      xaxis=dict(title='Heure', tickvals=list(range(24)), ticktext=[f'{hour}:00' for hour in range(24)],tickangle=-90),
                      yaxis=dict(title='Frequency'),
                      bargap=0.1)

    fig.update_xaxes(title_text='Heure', tickvals=list(range(24)), ticktext=[f'{hour}:00' for hour in range(24)],tickangle=-90, row=1, col=2)

    fig.show()


### Distribution of day type by prestation

In [47]:
grouped = JAN24_df.groupby('Prestation').size().sort_values(ascending=False)

prestation_trace = go.Bar(
    x=grouped.index,
    y=grouped.values,
    name='Count',
    marker=dict(color='orchid')
)

data = [prestation_trace]

layout = go.Layout(
    title='Distribution of Prestation',
    xaxis=dict(
        title='Prestation',
        tickangle=-90,
        tickfont=dict(
            size=10
        )
    ),
    yaxis=dict(
        title='Count'
    )
)

fig = go.Figure(data=data, layout=layout)

pio.show(fig)


In [7]:
grouped = JAN24_df.groupby(['Prestation', 'Day type']).size().unstack(fill_value=0)

weekday_trace = go.Bar(
    x=grouped.index,
    y=grouped['Week-day'],
    name='Week-day',
    marker=dict(color='orchid')
)
weekend_trace = go.Bar(
    x=grouped.index,
    y=grouped['Week-end'],
    name='Week-end',
    marker=dict(color='mediumorchid')
)

data = [weekday_trace, weekend_trace]

layout = go.Layout(
    title='Distribution of Day Type for each Prestation',
    xaxis=dict(
        title='Prestation',
        tickangle=-90,
        tickfont=dict(
            size=10
        )
    ),
    yaxis=dict(
        title='Count'
    ),
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)

pio.show(fig)


### Mean duration by prestation 

In [8]:
mean_duration = JAN24_df.groupby('Prestation')['Duration'].mean().sort_values()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=mean_duration.index,  # 'Prestation' as x-axis label
    y=mean_duration.values,  # Mean duration as y-axis label
    text=mean_duration.values.round(0),  # Display mean duration on top of the bars, rounded to 2 decimal places
    textposition='auto',  # Automatically position the text on top of the bars
    marker=dict(color='plum'),  # Set the color of the bars
))

fig.update_layout(
    title='Mean Duration by Prestation',
    xaxis=dict(title='Prestation'),  # Set x-axis label
    yaxis=dict(title='Mean Duration (minutes)'),  # Set y-axis label
    bargap=0.2,  # Set the gap between bars
    showlegend=False  # Hide legend as we have only one trace
)

fig.show()

# Clustering / segmenting clients

In [9]:
df = JAN24_df[['Day type', 'Day time', 'Prestation', 'Duration']]

df = pd.get_dummies(df, columns=['Day type', 'Day time', 'Prestation'])

scaler = StandardScaler()
numerical_columns = ['Duration']  # Add other numerical columns if any
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])


k = 5  # You can choose an appropriate number based on domain knowledge or clustering evaluation metrics

kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(df)

cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=df.columns[:-1])  # Excluding 'cluster' column

cluster_sizes = df['cluster'].value_counts()

  File "c:\Users\dell\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 217, in _count_physical_cores
    raise ValueError(


In [10]:
cluster_sizes

cluster
3    871
2    745
0    509
1    394
4    244
Name: count, dtype: int64

### Geographic distribution and density

In [11]:
map_center = [clients_df['Latitude'].mean(), clients_df['Longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

for index, row in clients_df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=f"Latitude: {row['Latitude']}, Longitude: {row['Longitude']}").add_to(mymap)

mymap.save("client_map.html")


In [12]:
import folium

map_center = [clients_df['Latitude'].mean(), clients_df['Longitude'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

for index, row in clients_df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=f"Client Location: {row['Latitude']}, {row['Longitude']}", icon=folium.Icon(color='blue')).add_to(mymap)

for index, row in intervenants_df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=f"Intervenant Location: {row['Latitude']}, {row['Longitude']}", icon=folium.Icon(color='red')).add_to(mymap)

#mymap.save("combined_map.html")

In [13]:

# Adding legend
legend_html = """
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 150px; height: 90px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">
     <p><strong>Legend</strong></p>
     <p><i class="fa fa-circle fa-1x" style="color:blue"></i> Client</p>
     <p><i class="fa fa-circle fa-1x" style="color:red"></i> Intervenant</p>
      </div>
     """
mymap.get_root().html.add_child(folium.Element(legend_html))

In [14]:
mymap

In [15]:
import pandas as pd
import folium
from folium.plugins import HeatMap

# Load the DataFrame with client locations (assuming it's already loaded)

# Calculate client density based on longitude and latitude
density_df = clients_df.groupby(['Longitude', 'Latitude']).size().reset_index(name='Density')

# Convert density data to list of [lat, lon, weight]
density_data = [[row['Latitude'], row['Longitude'], row['Density']] for index, row in density_df.iterrows()]

# Create a map centered around Paris
map_paris = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Add heat map layer
HeatMap(density_data, radius=15).add_to(map_paris)

# Save the map as an HTML file
map_paris.save('client_density_heatmap.html')

# Display the map
map_paris


### Frequency analysis

#### Global, including all prestations

In [16]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

orders_count = JAN24_df.groupby('ID Client')['Prestation'].count().reset_index()
orders_count = orders_count.rename(columns={'Prestation': 'Orders Count'})

fig = px.histogram(orders_count, x='Orders Count', title='Distribution of Orders Count Across Clients - all prestations',color_discrete_sequence=['darkorchid'])
fig.show()


#### Zoom on smaller frequencies to identify sporadic needs

In [17]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

orders_count = JAN24_df.groupby('ID Client')['Prestation'].count().reset_index()
orders_count = orders_count.rename(columns={'Prestation': 'Orders Count'})

filtered_orders_count = orders_count[(orders_count['Orders Count'] >= 0) & (orders_count['Orders Count'] <= 9)]

fig = px.histogram(filtered_orders_count, x='Orders Count', title='Distribution of Orders Count Across Clients (Count between 0 and 9)', color_discrete_sequence=['darkorchid'])
fig.show()

#### Frequency by prestation

In [18]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

# Group by 'ID Client' and count the number of orders for each 'Prestation'
orders_count = JAN24_df.groupby(['ID Client', 'Prestation']).size().reset_index(name='Orders Count')

# Plot distribution of orders count across clients for each specified 'Prestation' using Plotly with darkorchid color
fig = px.histogram(orders_count, x='Orders Count', facet_col='Prestation', title='Distribution of Orders Count Across Clients for Each Prestation', color_discrete_sequence=['darkorchid'], facet_col_wrap=3)
fig.show()

In [19]:
orders_per_client = JAN24_df.groupby('ID Client').size().reset_index(name='Orders Count')

clients_with_single_order = orders_per_client[orders_per_client['Orders Count'] == 1]

percentage_single_order_clients = (len(clients_with_single_order) / len(orders_per_client)) * 100

print(f"Percentage of clients who placed only one order in the month: {percentage_single_order_clients:.2f}%")

Percentage of clients who placed only one order in the month: 5.22%


In [20]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

JAN24_df['Week'] = JAN24_df['Date'].dt.isocalendar().week
JAN24_df['Month'] = JAN24_df['Date'].dt.month

weekly_counts_per_month = JAN24_df.groupby(['Month', 'ID Client'])['Week'].nunique()


In [21]:
percentage_reg = weekly_counts_per_month[weekly_counts_per_month >= 4].size / weekly_counts_per_month.size

print(f"Percentage of clients who placed at least one order every week: {percentage_reg*100:.2f}%")

Percentage of clients who placed at least one order every week: 73.91%


### Regular clients

In [22]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

orders_per_week = JAN24_df.groupby(['ID Client', JAN24_df['Date'].dt.isocalendar().week])['Prestation'].count().reset_index()

clients_with_same_orders_per_week = orders_per_week.groupby('ID Client').filter(lambda x: len(x['Prestation'].unique()) == 1)

clients_list = clients_with_same_orders_per_week['ID Client'].unique()

print("Clients with the same number of orders every week:")
print(clients_list)

Clients with the same number of orders every week:
[  78691389   78873790   79598982   79600570   82135699   88672872
   97922195  167937489  188178602  197278699  205495806  297413132
  298597763  334442311  334811042  352744112  433149947  438309694
  452687307  452919640  474721965  476321344  477580072  479391393
  481413999  531956083  565078696  569981306  570590313  599471591
  607763952  609495444  657893693  662280369  727679560  728081901
  740412946  751167409  755498685  756396600  764206151  793803218
  817947464  826666511  827142453  855391156  858147751  860937824
  863374497 1452740690 1452763176 1453084257]


In [23]:
JAN24_df['Date'] = pd.to_datetime(JAN24_df['Date'])

orders_per_week = JAN24_df.groupby(['ID Client', JAN24_df['Date'].dt.isocalendar().week])['Prestation'].count().reset_index()

clients_with_same_orders_per_week = orders_per_week.groupby('ID Client').filter(lambda x: len(x['Prestation'].unique()) == 1)

num_clients_same_orders = len(clients_with_same_orders_per_week['ID Client'].unique())

total_unique_clients = JAN24_df['ID Client'].nunique()

percentage_clients_same_orders = (num_clients_same_orders / total_unique_clients) * 100

print("Percentage of clients with the same number of orders every week:", percentage_clients_same_orders)

Percentage of clients with the same number of orders every week: 45.21739130434783


In [24]:
reg_client1 = JAN24_df[JAN24_df['ID Client'] == 78691389]

In [25]:
reg_client2 = JAN24_df[JAN24_df['ID Client'] == 662280369]

In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=reg_client1['Date'], y=[1] * len(reg_client1), mode='markers', marker=dict(color='plum', size=10)))

# Update layout
fig.update_layout(
    title='Prestation orders for regular client (id = 78691389)',
    xaxis_title='Date',
    yaxis=dict(visible=False),  # Hide y-axis labels
    xaxis=dict(tickangle=45),  # Rotate x-axis labels
    margin=dict(l=0, r=0, t=50, b=0),  # Adjust margins
    height=200,  # Adjust height
    width=600  # Adjust width
)

fig.show()


In [27]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=reg_client2['Date'], y=[1] * len(reg_client2), mode='markers', marker=dict(color='plum', size=10)))

# Update layout
fig.update_layout(
    title='Prestation orders for regular clients (id = 662280369)',
    xaxis_title='Date',
    yaxis=dict(visible=False),  # Hide y-axis labels
    xaxis=dict(tickangle=45),  # Rotate x-axis labels
    margin=dict(l=0, r=0, t=50, b=0),  # Adjust margins
    height=200,  # Adjust height
    width=600  # Adjust width
)

fig.show()

In [28]:

filtered_df = JAN24_df[JAN24_df['ID Client'].isin(clients_list)]

prestation_counts = filtered_df['Prestation'].value_counts()

fig = px.bar(prestation_counts, x=prestation_counts.index, y=prestation_counts.values, color_discrete_sequence=['darkorchid'])

fig.update_layout(
    title='Distribution of Prestation for Regular Clients',
    xaxis_title='Prestation',
    yaxis_title='Frequency',
    xaxis={'categoryorder':'total descending'},  # Sort x-axis labels by frequency
    xaxis_tickangle=0  # Rotate x-axis labels for better readability
)

fig.show()


## Nurse profiling

In [29]:
total_intervenants = intervenants_df.shape[0]

# Calculate the number of intervenants with a car ('Oui' in 'Véhicule Personnel' column)
intervenants_with_car = intervenants_df[intervenants_df['Véhicule personnel'] == 'Oui'].shape[0]

# Calculate the percentage of intervenants with a car
percentage_with_car = (intervenants_with_car / total_intervenants) * 100

print("Percentage of intervenants with a car: {:.2f}%".format(percentage_with_car))


Percentage of intervenants with a car: 79.17%


In [30]:
total_intervenants = intervenants_df.shape[0]

# Calculate the number of intervenants with a car ('Oui' in 'Véhicule Personnel' column)
intervenants_with_permis = intervenants_df[intervenants_df['Permis'] == 'Oui'].shape[0]

# Calculate the percentage of intervenants with a car
percentage_with_permis = (intervenants_with_permis / total_intervenants) * 100

print("Percentage of intervenants with permis: {:.2f}%".format(percentage_with_permis))

Percentage of intervenants with permis: 79.17%


In [31]:
mean_available = intervenants_df['Total available days '].mean()
print("Average monthly availability in days: {:.2f}".format(mean_available))

Average monthly availability in days: 18.00


In [32]:
intervenants_df_trimmed = intervenants_df.iloc[:, 8:]

transposed_df = intervenants_df_trimmed.T

In [33]:
# Calculate the sum along the rows (axis=1) and create a bar plot
fig = go.Figure(go.Bar(
    x=transposed_df.index,  # Index of transposed DataFrame
    y=transposed_df.sum(axis=1),  # Sum along rows
    marker_color='orchid'  # Color of bars
))

# Customize layout
fig.update_layout(
    title='Available service providers in the month',
    xaxis_title='',
    yaxis_title='Number of intervenants',
    template='plotly_white'  # Theme
)

# Show the plot
fig.show()


In [34]:
client_dist = pd.read_csv('client_client_distance_v1.csv')

In [35]:
JAN24_df = JAN24_df.sort_values(by=['ID Intervenant', 'Date'])
JAN24_df['Previous Client'] = JAN24_df.groupby('ID Intervenant')['ID Client'].shift(1)

In [36]:
merged_df = JAN24_df.merge(client_dist, how='left', left_on=['ID Client', 'Previous Client'], right_on=['ID Client 1', 'ID Client 2'])
merged_df.drop(['ID Client 1', 'ID Client 2'], axis=1, inplace=True)
merged_df.rename(columns={'Duration_Car': 'Duration_Car_Previous', 'Duration_Bike': 'Duration_Bike_Previous'}, inplace=True)

In [37]:
merged_df = merged_df.merge(intervenants_df[['ID Intervenant', 'Permis']], how='left', on='ID Intervenant')

In [38]:
def convert_duration(duration_str):
    parts = duration_str.split()
    hours = 0
    mins = 0
    for i in range(len(parts)):
        if 'hour' in parts[i]:
            hours = int(parts[i-1])
        elif 'min' in parts[i]:
            mins = int(parts[i-1])
    return hours * 60 + mins

In [39]:
merged_df['Duration_Car_Previous'] = merged_df['Duration_Car_Previous'].astype(str)
merged_df['Duration_Bike_Previous'] = merged_df['Duration_Bike_Previous'].astype(str)

In [40]:
merged_df['Duration_Car_Previous'] = merged_df['Duration_Car_Previous'].apply(convert_duration)
merged_df['Duration_Bike_Previous'] = merged_df['Duration_Bike_Previous'].apply(convert_duration)

In [41]:
summed_df = merged_df.groupby(['Date', 'ID Intervenant'])[['Duration_Car_Previous', 'Duration_Bike_Previous']].sum().reset_index()

In [42]:
summed_df = summed_df.merge(intervenants_df[['ID Intervenant', 'Permis']], how='left', on='ID Intervenant')

In [43]:
summed_df_bike = summed_df[summed_df['Permis']=='Non']
summed_df_car = summed_df[summed_df['Permis']=='Oui']

In [44]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=summed_df_bike['Duration_Car_Previous'],
                            xbins=dict(
                                start=summed_df['Duration_Car_Previous'].min(),
                                end=summed_df['Duration_Car_Previous'].max(),
                                size=15),
                            marker_color='rebeccapurple',
                            opacity=0.75))

mean_duration = np.mean(summed_df['Duration_Car_Previous'])

fig.add_vline(x=mean_duration, line=dict(color='darkmagenta', width=2, dash='dash'))

fig.update_layout(title_text='Daily commuting time by bike distribution (all time all intervenants)',
                  xaxis_title='Daily commuting time (mins)',
                  yaxis_title='Frequency')

fig.show()

In [45]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=summed_df_car['Duration_Car_Previous'],
                            xbins=dict(
                                start=summed_df['Duration_Car_Previous'].min(),
                                end=summed_df['Duration_Car_Previous'].max(),
                                size=15),
                            marker_color='rebeccapurple',
                            opacity=0.75))

mean_duration = np.mean(summed_df['Duration_Car_Previous'])

fig.add_vline(x=mean_duration, line=dict(color='darkmagenta', width=2, dash='dash'))

fig.update_layout(title_text='Daily commuting time by car distribution (all time all intervenants)',
                  xaxis_title='Daily commuting time (mins)',
                  yaxis_title='Frequency')

fig.show()